Python Code for configuring the shapefile to be run in SUMMA and Mizuroutie

There are 3 files that are needed:
- Catchment File
  - Required Fields Include: GRU_ID, HRU_ID, center_lat, center_lon, HRU_area (m^2)
- River Network File
  - Required Fields Include: COMID, NextDownID, slope, length
- Routing Basin File
  - Required Fields Include: COMID, area, hru_to_seg

First we want to verify that the COMID's match in the files we generated from camels_spat match

In [1]:
import geopandas as gpd

In [2]:
# We generated two files in the camels_spat workflow: basin and river shapefiles.

basin_file = "/Users/kyleklenk/ProgrammingProjects/OpenWQ-Projects/CAMELS_spat_data/OpenWQ_Basins/Great_Slave_Lake_Massive/shapefiles/distributed/CAN_06JB001_distributed_basin.shp"
river_file = "/Users/kyleklenk/ProgrammingProjects/OpenWQ-Projects/CAMELS_spat_data/OpenWQ_Basins/Great_Slave_Lake_Massive/shapefiles/distributed/CAN_06JB001_distributed_river.shp"

basin = gpd.read_file(basin_file)
river = gpd.read_file(river_file)

# Find there exists the same COMID in both files
basin_comids = basin["COMID"].tolist()
river_comids = river["COMID"].tolist()

# Check if they are the same length
print(len(basin_comids))
print(len(river_comids))
if len(basin_comids) != len(river_comids):
    print("Error: basin and river shapefiles do not have the same number of COMIDs.")

for comid in basin_comids:
    if comid not in river_comids:
        print("Error: COMID {} in basin shapefile but not in river shapefile.".format(comid))

# if you see any ERROR Output above, then you need to fix the shapefiles.

3515
3551
Error: basin and river shapefiles do not have the same number of COMIDs.


#### Format the catchment file

In [3]:
# Set GRU_ID to the COMID
basin.rename(columns={"COMID" : "GRU_ID"}, inplace=True)

# Assign an HRU_ID to each row
basin.insert(loc=1, column="HRU_ID", value=range(1, len(basin) + 1))

# Convert units of area from square kilometers to square meters
basin.rename(columns={"unitarea" : "HRU_area"}, inplace=True)
basin["HRU_area"] = basin["HRU_area"] * 1000

basin.insert(loc=2, column="center_lat", value= -99)
basin.insert(loc=3, column="center_lon", value= -99)

# get center lat/lon for each basin
for index, row in basin.iterrows():
    center_point = row.geometry.centroid
    basin.at[index, "center_lat"] = center_point.y
    basin.at[index, "center_lon"] = center_point.x

# Save the file to a new shapefile
basin.to_file("/Users/kyleklenk/ProgrammingProjects/OpenWQ-Projects/CAMELS_spat_data/OpenWQ_Basins/Athabasca_River/required/catchment/athabasca_river_distributed_basin.shp")

#### Format the river network file

In [4]:

# Remove the unnecessary columns
river_network = river[["COMID", "NextDownID", "slope", "lengthkm", "geometry"]]
river_network.rename(columns={"lengthkm" : "length"}, inplace=True)

# Convert units of length from kilometers to meters
river_network["length"] = river_network["length"] * 1000

# Save the file to a new shapefile
river_network.to_file("/Users/kyleklenk/ProgrammingProjects/OpenWQ-Projects/CAMELS_spat_data/OpenWQ_Basins/Athabasca_River/required/river_network/athabasca_river_network.shp")




/var/folders/g6/6224gnj95j3cdty1v4ry58b40000gp/T/ipykernel_95442/3936320750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  river_network.rename(columns={"lengthkm" : "length"}, inplace=True)
/Users/kyleklenk/ProgrammingProjects/OpenWQ-Projects/CAMELS_spat_data/camels-spat-env/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


#### Format the routing basin file

In [5]:
print(basin.head())
river_basin = basin[["GRU_ID", "HRU_area", "geometry"]]
hru_seg = basin[["GRU_ID"]]


hru_seg.rename(columns={"GRU_ID" : "hru_to_seg"}, inplace=True)

# # add the column to become the 3rd column
river_basin.insert(loc=2, column="hru_to_seg", value=hru_seg)
river_basin.rename(columns={"HRU_area" : "area"}, inplace=True)
river_basin.rename(columns={"GRU_ID" : "COMID"}, inplace=True)

print(river_basin.head())

# # Save the file to a new shapefile
river_basin.to_file("/Users/kyleklenk/ProgrammingProjects/OpenWQ-Projects/CAMELS_spat_data/OpenWQ_Basins/Athabasca_River/required/river_basins/athabasca_river_basins.shp")


     GRU_ID  HRU_ID  center_lat  center_lon      HRU_area  \
0  82029834       1   58.392413 -110.899948  30318.528075   
1  82029835       2   58.449079 -110.997893  29935.896248   
2  82029836       3   58.445046 -111.168171  21379.419558   
3  82029837       4   58.436077 -111.375988  32454.539642   
4  82029838       5   58.380760 -111.522224   2172.137820   

                                            geometry  
0  MULTIPOLYGON (((-110.90042 58.42208, -110.9004...  
1  POLYGON ((-111.01375 58.49375, -111.01375 58.4...  
2  MULTIPOLYGON (((-111.14042 58.47042, -111.1404...  
3  POLYGON ((-111.26208 58.46125, -111.26208 58.4...  
4  POLYGON ((-111.52125 58.39208, -111.52125 58.3...  
      COMID          area  hru_to_seg  \
0  82029834  30318.528075    82029834   
1  82029835  29935.896248    82029835   
2  82029836  21379.419558    82029836   
3  82029837  32454.539642    82029837   
4  82029838   2172.137820    82029838   

                                            geometry  
0

/var/folders/g6/6224gnj95j3cdty1v4ry58b40000gp/T/ipykernel_95442/403447173.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hru_seg.rename(columns={"GRU_ID" : "hru_to_seg"}, inplace=True)
/var/folders/g6/6224gnj95j3cdty1v4ry58b40000gp/T/ipykernel_95442/403447173.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  river_basin.rename(columns={"HRU_area" : "area"}, inplace=True)
/var/folders/g6/6224gnj95j3cdty1v4ry58b40000gp/T/ipykernel_95442/403447173.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/panda